<a href="https://colab.research.google.com/github/Yash1289/Book-Recommender-System/blob/main/Book_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
import numpy as np
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
books_path = "/content/drive/MyDrive/Colab Notebooks/data_book_recommendation/Books.csv"
ratings_path = "/content/drive/MyDrive/Colab Notebooks/data_book_recommendation/Ratings.csv"
users_path = "/content/drive/MyDrive/Colab Notebooks/data_book_recommendation/Users.csv"

In [4]:
books_df = pd.read_csv(books_path , low_memory=False)
ratings_df = pd.read_csv(ratings_path , low_memory = False)
users_df = pd.read_csv(users_path , low_memory = False)

In [5]:
books_df.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [6]:
ratings_df.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [7]:
users_df.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [8]:
ratings_df = ratings_df.groupby("ISBN").filter(lambda x : len(x) >= 5)
ratings_df["ISBN"].value_counts().tail()

074324527X    5
0802135838    5
0140052216    5
0553583719    5
006018485X    5
Name: ISBN, dtype: int64

In [9]:
ratings_df["ISBN"].nunique()

43765

In [10]:
#df.groupby("A").filter(lambda x: len(x) > 1)

In [11]:
print(books_df.shape)
print(ratings_df.shape)
print(users_df.shape)

(271360, 8)
(699055, 3)
(278858, 3)


In [12]:
ratings_df[ratings_df.duplicated(subset = ["User-ID" , "ISBN"])]

,User-ID,ISBN,Book-Rating


In [13]:
ratings_df["RatingCat"] = ratings_df["Book-Rating"].apply(lambda x : 5 if x == 0 else x)

In [14]:
ratings_df["RatingCat"].value_counts()

5     463856
8      65669
10     47144
7      46151
9      43616
6      21575
4       5117
3       3398
2       1586
1        943
Name: RatingCat, dtype: int64

In [15]:
books_df[books_df.duplicated(subset = ["Book-Title"])]

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
239,0743486226,Angels &amp; Demons,Dan Brown,2003,Atria,http://images.amazon.com/images/P/0743486226.0...,http://images.amazon.com/images/P/0743486226.0...,http://images.amazon.com/images/P/0743486226.0...
355,0140374248,James and the Giant Peach,Roald Dahl,2000,Penguin USA (Paper),http://images.amazon.com/images/P/0140374248.0...,http://images.amazon.com/images/P/0140374248.0...,http://images.amazon.com/images/P/0140374248.0...
405,006101351X,The Perfect Storm : A True Story of Men Agains...,Sebastian Junger,1998,HarperTorch,http://images.amazon.com/images/P/006101351X.0...,http://images.amazon.com/images/P/006101351X.0...,http://images.amazon.com/images/P/006101351X.0...
456,044021145X,The Firm,John Grisham,1992,Bantam Dell Publishing Group,http://images.amazon.com/images/P/044021145X.0...,http://images.amazon.com/images/P/044021145X.0...,http://images.amazon.com/images/P/044021145X.0...
475,014023828X,The Tortilla Curtain,T. Coraghessan Boyle,1996,Penguin Books,http://images.amazon.com/images/P/014023828X.0...,http://images.amazon.com/images/P/014023828X.0...,http://images.amazon.com/images/P/014023828X.0...
...,...,...,...,...,...,...,...,...
271246,0195124995,For Cause and Comrades: Why Men Fought in the ...,James M. McPherson,1998,Oxford University Press,http://images.amazon.com/images/P/0195124995.0...,http://images.amazon.com/images/P/0195124995.0...,http://images.amazon.com/images/P/0195124995.0...
271315,1902852036,All the Queen's Men,Nick Elwood,1999,Gay Men's Press,http://images.amazon.com/images/P/1902852036.0...,http://images.amazon.com/images/P/1902852036.0...,http://images.amazon.com/images/P/1902852036.0...
271316,0441011799,Singularity Sky,Charles Stross,2004,Ace,http://images.amazon.com/images/P/0441011799.0...,http://images.amazon.com/images/P/0441011799.0...,http://images.amazon.com/images/P/0441011799.0...
271346,0395264707,Dreamsnake,Vonda N. McIntyre,1978,Houghton Mifflin,http://images.amazon.com/images/P/0395264707.0...,http://images.amazon.com/images/P/0395264707.0...,http://images.amazon.com/images/P/0395264707.0...


In [16]:
ratings_df["User-ID"].nunique()

78326

In [17]:
users_df.shape

(278858, 3)

In [18]:
filteredUsers_df = users_df[(users_df["Age"] <= 100) | (users_df["Age"].isna())]

In [19]:
filteredUsers_df.shape

(278492, 3)

In [20]:
books_ratings_df = pd.merge(ratings_df , books_df , on = ["ISBN"])

In [21]:
books_ratings_df["User-ID"].nunique()

74206

In [22]:
books_ratings_df[books_ratings_df["ISBN"].str.len() != 10 ]

,User-ID,ISBN,Book-Rating,RatingCat,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L


In [23]:
users_books_ratings_df = pd.merge(books_ratings_df , filteredUsers_df , on =["User-ID"])
users_books_ratings_df["User-ID"].nunique()

74114

In [24]:
users_books_ratings_df.head()

,User-ID,ISBN,Book-Rating,RatingCat,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276725,034545104X,0,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,2313,034545104X,5,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0
2,2313,0812533550,9,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,"cincinnati, ohio, usa",23.0
3,2313,0679745580,8,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,"cincinnati, ohio, usa",23.0
4,2313,0060173289,9,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,"cincinnati, ohio, usa",23.0


In [25]:
users_ratedBooks_count_df = users_books_ratings_df.groupby(['User-ID', 'ISBN']).size().groupby('User-ID').size()
print('# of users: %d' % len(users_ratedBooks_count_df))

users_with_enough_ratedBooks_df = users_ratedBooks_count_df[users_ratedBooks_count_df >= 5].reset_index()[['User-ID']]
print('# of users with at least 5 interactions: %d' % len(users_with_enough_ratedBooks_df))

# of users: 74114
# of users with at least 5 interactions: 15722


In [26]:
print('# of interactions: %d' % len(users_books_ratings_df))
ratings_from_selected_users_df = users_books_ratings_df.merge(users_with_enough_ratedBooks_df, 
               how = 'right',
               left_on = 'User-ID',
               right_on = 'User-ID')
print('# of interactions from users with at least 5 interactions: %d' % len(ratings_from_selected_users_df))

# of interactions: 668505
# of interactions from users with at least 5 interactions: 582021


In [27]:
ratings_from_selected_users_df["User-ID"].value_counts().tail()

245765    5
54142     5
245790    5
54130     5
228442    5
Name: User-ID, dtype: int64

In [28]:
ratings_from_selected_users_df[ratings_from_selected_users_df.duplicated( subset = ["User-ID" , "Book-Title"], keep= "last")]

,User-ID,ISBN,Book-Rating,RatingCat,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
131,254,0439064864,9,9,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439064864.0...,http://images.amazon.com/images/P/0439064864.0...,http://images.amazon.com/images/P/0439064864.0...,"minneapolis, minnesota, usa",24.0
145,254,0439136369,9,9,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,2001,Scholastic,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,http://images.amazon.com/images/P/0439136369.0...,"minneapolis, minnesota, usa",24.0
155,254,0380789019,0,5,Neverwhere,Neil Gaiman,1998,Avon,http://images.amazon.com/images/P/0380789019.0...,http://images.amazon.com/images/P/0380789019.0...,http://images.amazon.com/images/P/0380789019.0...,"minneapolis, minnesota, usa",24.0
257,254,037582345X,9,9,"The Golden Compass (His Dark Materials, Book 1)",PHILIP PULLMAN,2002,Knopf Books for Young Readers,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,http://images.amazon.com/images/P/037582345X.0...,"minneapolis, minnesota, usa",24.0
258,254,0375823468,9,9,"The Subtle Knife (His Dark Materials, Book 2)",PHILIP PULLMAN,2002,Knopf Books for Young Readers,http://images.amazon.com/images/P/0375823468.0...,http://images.amazon.com/images/P/0375823468.0...,http://images.amazon.com/images/P/0375823468.0...,"minneapolis, minnesota, usa",24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
580925,278418,0399137734,0,5,Dragon Tears,Dean R. Koontz,1993,Putnam Pub Group,http://images.amazon.com/images/P/0399137734.0...,http://images.amazon.com/images/P/0399137734.0...,http://images.amazon.com/images/P/0399137734.0...,"omaha, nebraska, usa",NaN
581056,278418,0446516317,0,5,The Witness,Sandra Brown,1995,Replica Books,http://images.amazon.com/images/P/0446516317.0...,http://images.amazon.com/images/P/0446516317.0...,http://images.amazon.com/images/P/0446516317.0...,"omaha, nebraska, usa",NaN
581075,278418,0446604275,0,5,Primary Colors: A Novel of Politics,Anonymous,1996,Warner Books,http://images.amazon.com/images/P/0446604275.0...,http://images.amazon.com/images/P/0446604275.0...,http://images.amazon.com/images/P/0446604275.0...,"omaha, nebraska, usa",NaN
581138,278418,0451181379,0,5,The Door to December,Dean R. Koontz,1994,Signet Book,http://images.amazon.com/images/P/0451181379.0...,http://images.amazon.com/images/P/0451181379.0...,http://images.amazon.com/images/P/0451181379.0...,"omaha, nebraska, usa",NaN


In [29]:
implicit_duplicates = ratings_from_selected_users_df[ratings_from_selected_users_df.duplicated( subset = ["User-ID" , "Book-Title"], keep=False) & (ratings_from_selected_users_df["Book-Rating"] == 0) ]
implicit_duplicates = implicit_duplicates.sort_values(by = ["Year-Of-Publication"] , ascending = False)
implicit_duplicates.drop_duplicates(subset = ["User-ID" , "Book-Title"] , keep = "last" , inplace = True)

ratings_from_selected_users_df.drop(implicit_duplicates.index , inplace = True)
#k = l[(l["User-ID"] == 254)]
#k[(k.duplicated( subset = ["Book-Title"] , keep = False))]

In [30]:
selected_ratings_df = ratings_from_selected_users_df.sort_values(by =["Year-Of-Publication"] , ascending = False ) 

print(len(selected_ratings_df[selected_ratings_df.duplicated(subset = ["User-ID" , "Book-Title"] , keep = "first")]))

selected_ratings_df.drop_duplicates(subset = ["User-ID" , "Book-Title"] , keep = "first" , inplace = True)

688


In [31]:
selected_ratings_df.shape

(578600, 13)

In [32]:
selected_ratings_df.head()

,User-ID,ISBN,Book-Rating,RatingCat,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
502335,240144,0140301690,0,5,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,"muskego, wisconsin, usa",34.0
172225,82831,0140301690,7,7,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,"nashville, tennessee, usa",21.0
91286,41700,0140301690,7,7,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,"kerzers, fribourg, switzerland",42.0
88264,40029,0140301690,10,10,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,"lausanne, vaud, switzerland",26.0
159440,76626,0140301690,0,5,Alice's Adventures in Wonderland and Through t...,Lewis Carroll,2050,Puffin Books,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,http://images.amazon.com/images/P/0140301690.0...,"london, england, united kingdom",38.0


In [33]:
from sklearn.model_selection import train_test_split

In [34]:
book_ratings_train_df, book_ratings_test_df = train_test_split(selected_ratings_df,
                                   stratify = selected_ratings_df['User-ID'], 
                                   test_size=0.20,
                                   random_state=42)

print('# interactions on Train set: %d' % len(book_ratings_train_df))
print('# interactions on Test set: %d' % len(book_ratings_test_df))

# interactions on Train set: 462880
# interactions on Test set: 115720


Filter the duplicate book reads earlier before filtering the dataframe to only have users who have read more than or equal to 5 books

In [45]:
from scipy.sparse import csr_matrix, lil_matrix

In [49]:
users = book_ratings_train_df['User-ID'].unique()
books = book_ratings_train_df['Book-Title'].unique()

In [50]:
users.shape[0]

15722

In [51]:
books.shape[0]

36150

# New Section

In [55]:
from tqdm import tqdm
tqdm.pandas()

In [57]:
book_ratings_train_df['books'] = book_ratings_train_df['Book-Title'].progress_apply(lambda x : np.argwhere(books == x)[0][0])

100%|██████████| 462880/462880 [16:49<00:00, 458.33it/s]


In [71]:
book_ratings_train_df['users'] = book_ratings_train_df['User-ID'].apply(lambda x : np.argwhere(users == x)[0][0])

In [72]:
from google.colab import files

In [73]:
book_ratings_train_df.to_csv("file1.csv")

In [74]:
files.download("file1.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [75]:
occurences = lil_matrix((users.shape[0], books.shape[0]), dtype='int8')

def set_occurences(visitor, item , rating):
    occurences[visitor, item] = rating

book_ratings_train_df.apply(lambda row: set_occurences(row['users'], row['books'] , row["RatingCat"]), axis=1)

occurences

<15722x36150 sparse matrix of type '<class 'numpy.int8'>'
	with 462880 stored elements in List of Lists format>

In [ ]:
user_ids = list(book_ratings_train_df["users"].unique())
book_ids = list(book_ratings_train_df["books"].unique())

In [ ]:
occurences

NameError: ignored

In [101]:
class MF():
    
    np.seterr(all='raise')

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K)).round(3)
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K)).round(3)
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        i,j = self.R.nonzero()
        values = self.R[self.R.nonzero()]
        self.samples = list(zip(i , j, values))
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1)%2 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = round((r - prediction),3)

            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += (self.alpha * ((e * self.Q[j, :]).round(3) - (self.beta * self.P[i,:]).round(3))).round(3)
            self.Q[j, :] += (self.alpha * ((e * P_i).round(3) - (self.beta * self.Q[j,:]).round(3))).round(3)

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)

In [102]:
R = occurences.toarray()

mf = MF(R, K=20, alpha=0.01, beta=0.001, iterations=60)
training_process = mf.train()
print()
print("P x Q:")
print(mf.full_matrix())

Iteration: 2 ; error = 1004.4701
Iteration: 4 ; error = 968.4807
Iteration: 6 ; error = 933.0264
Iteration: 8 ; error = 888.4698
Iteration: 10 ; error = 835.3564
Iteration: 12 ; error = 781.8942
Iteration: 14 ; error = 731.0959
Iteration: 16 ; error = 683.8881
Iteration: 18 ; error = 640.5845
Iteration: 20 ; error = 601.1671
Iteration: 22 ; error = 565.4052
Iteration: 24 ; error = 533.1534
Iteration: 26 ; error = 504.0402
Iteration: 28 ; error = 477.9522
Iteration: 30 ; error = 454.5708
Iteration: 32 ; error = 433.5826
Iteration: 34 ; error = 414.9711
Iteration: 36 ; error = 398.2045
Iteration: 38 ; error = 383.2288
Iteration: 40 ; error = 369.9399
Iteration: 42 ; error = 357.9299
Iteration: 44 ; error = 347.1950
Iteration: 46 ; error = 337.6481
Iteration: 48 ; error = 329.0009
Iteration: 50 ; error = 321.1710
Iteration: 52 ; error = 314.0976
Iteration: 54 ; error = 307.6601
Iteration: 56 ; error = 301.8630
Iteration: 58 ; error = 296.5873
Iteration: 60 ; error = 291.6560

P x Q:
[[5.0

In [103]:
mf.full_matrix().shape

(15722, 36150)

In [105]:
user_ids = list(book_ratings_train_df["User-ID"].unique())
book_names = list(book_ratings_train_df["Book-Title"].unique())

In [107]:
cf_preds_df = pd.DataFrame(mf.full_matrix(), columns = book_names , index = user_ids)
cf_preds_df.head()

,Divine Secrets of the Ya-Ya Sisterhood: A Novel,Halloween Night,The Mammy,Every Mother's Nightmare (Onyx True Crime),The Matarese Circle,A Common Life : The Wedding Story (The Mitford Years),24 Hours,A Stockingful of Joy,The Servants of Twilight,Idiot (Penguin Classics),La Catilinaires,The Door to December,Little Women (Illustrated Junior Library),"The Shadow Academy (Star Wars: Young Jedi Knights, Book 2)",The Giver (21st Century Reference),P Is for Peril (Kinsey Millhone Mysteries (Hardcover)),The House of the Spirits,The Burglar in the Rye: A Bernie Rhodenbarr Mystery (Bernie Rhodenbarr Mysteries (Paperback)),If 2: 500 New Questions for the Game of Life,The Airship: The World of Teddy Ruxpin,The Shining,Bodyguard,"Eeyore, Be Happy (Little Golden Book)",Healing and the Mind,A Very Gothic Christmas (Holiday Classics),Watch Them Die,Tell No One,The Hundred Secret Senses,The PMS Outlaws : An Elizabeth MacPherson Novel,Judgment in Death,NEVERLAND,Talk Before Sleep: A Novel,The Companions : A Novel,Valor's Choice (Daw Book Collectors),Montana Sky,"The Accidental Buddhist: Mindfulness, Enlightenment, and Sitting Still, American Style","What Are Hyenas Laughing At, Anyway? An Imponderables Book",Postcards from the Underground: Portraits from the Beat Era,Night over Water,The Deceiver,...,Wanted!,Memoirs of a Survivor,The Englishman's Daughter: A True Story of Love and Betrayal in World War One,Tidewater Blood,In the Absence of the Sacred: The Failure of Technology and the Survival of the Indian Nations,Im Krebsgang: Eine Novelle,Bronsteins Kinder. Roman.,Amado amo (ColecciÃ³n Literatura),Ich tanze so schnell ich kann.,Killing Thyme (Wwl Mystery),Die Verwandlung.,"Murder Machine: A True Story of Murder, Madness, and the Mafia",The Clay Marble: And Related Readings (Literature Connections),Janice Gentle Gets Sexy,"Secrets of Fat-Free Baking: Over 130 Low-Fat &amp; Fat-Free Recipes for Scrumptious and Simple-To-Make Cakes, Cookies, Brownies, Muffins, Pies, Breads,",The Book of Revelation,I Giorni Dell'abbandono,Emergency!: True Stories from the Nation's Ers (Emergency!),Together Again: Twin Souls Reunite in Love and Life,Il Giro Di Boa,Indian Creek Chronicles : A Winter Alone in the Wilderness,La Testa Fra Le Nuvole,A Scientist Explores Spirit: A Biography of Emanuel Swedenborg With Key Concepts of His Theology,Cruel Miracles,Doble Juego,Graveyard: True Hauntings from an Old New England Cemetery,"Follow the Wind (Spanish Bit Saga Book, No 3)",Grandmaster,Diez negritos,Wintermond. Unheimlicher Roman.,Salmon Doubt,Coming of the Quantum Cats,Das Erbe des Zauberers. Ein Roman aus der bizarren Scheibenwelt,The Walls of Jericho (Ben Kamal and Danielle Barnea Novels (Paperback)),"Twelfth Night, Or, What You Will: With New and Updated Critical Essays and a Revised Bibliography (The Signet Classic Shakespeare)",Y2K. Der letzte Tag. Der Roman der Jahrtausendwende.,La Musica del Azar,Misadventures,The Boys from Brazil,Searching for Virginia Dare: A Fool's Errand
49980,5.003925,5.279624,7.161307,6.681722,5.655407,10.357403,6.441437,6.302616,4.253434,5.412540,6.063037,6.126493,5.647500,6.076855,9.111423,8.206735,8.562641,6.900887,7.196649,6.577652,8.976965,6.581390,5.797168,4.335642,5.613829,6.589569,6.608145,7.732066,9.720486,4.865797,5.434639,9.602438,6.880478,6.772435,7.781653,6.745869,9.390670,5.929517,5.063016,7.349097,...,5.805385,6.189054,7.618884,6.149530,5.648327,7.113271,5.221072,6.221305,6.530815,7.770732,6.093192,6.615489,5.250374,6.127022,6.148304,6.252250,6.119076,6.244541,6.161920,6.337876,6.552093,5.874579,6.804590,6.592786,6.075864,6.077136,5.767317,5.967869,7.574475,6.918554,5.792865,5.429684,6.742824,6.517585,7.122149,6.315948,6.383662,5.373903,6.318108,6.385408
24921,4.232258,4.827239,5.602037,5.265532,5.228051,5.899049,5.712648,5.425112,6.100265,4.724824,3.749302,5.168851,6.738892,5.490699,6.343105,5.758682,4.956382,5.017434,5.201271,4.934936,5.478076,5.188535,5.606770,4.174314,4.387316,4.756472,5.238388,5.165917,5.488399,4.9

In [ ]:
h = pd.DataFrame.sparse.from_spmatrix(occurences, columns = book_ids , index = user_ids)
h.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,36110,36111,36112,36113,36114,36115,36116,36117,36118,36119,36120,36121,36122,36123,36124,36125,36126,36127,36128,36129,36130,36131,36132,36133,36134,36135,36136,36137,36138,36139,36140,36141,36142,36143,36144,36145,36146,36147,36148,36149
0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,5,0,10,0,0,0,8,0,8,0,0,5,0,0,7,5,6,0,0,0,5,0,10,0,0,0,0,9,0,5,5,0,0,3,9,0,0,0,0,5,...,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,5,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
